In [1]:
from bs4 import BeautifulSoup
import requests
import re

# Web scraping from the radar AvroTros website
search_link = "https://radar.avrotros.nl/forum/energie-f51/index-s"

forum_url = BeautifulSoup(requests.get(search_link).content, 'html.parser')

page_index = 0

dict_topics = {}

for i in range(0, 100, 25):
    page_index = i

    forum_url = BeautifulSoup(requests.get(search_link + str(page_index) + '.html').content, 'html.parser')

    items = forum_url.find_all(class_=lambda x: x and x.startswith("row bg"))

    for item in items:
        title = item.find('a', class_='topictitle')
        topic_title = title.text

        topic_link = "https://radar.avrotros.nl/forum" + "/" + title['href'].lstrip('./..').split('.html')[0] + '.html'

        date_element = item.find("div", class_="topic-poster responsive-hide left-box").find("time")
        date = date_element.text

        code_link = topic_link[-11:]
        code = code_link[:6]

        reactions_element = item.find("dd", class_="posts")
        reactions = reactions_element.text
        # Remove the 'Reacties' text
        reactions = reactions[:-9]
        reactions = int(reactions)

        dict_topics[topic_title] = topic_link, date, code, reactions

# Convert dict to dataframe
import pandas as pd

df_topics = pd.DataFrame.from_dict(dict_topics, orient='index', columns=['link', 'date', 'code', 'reactions'])

df_topics

,link,date,code,reactions
De energiemarkt en salderen: hoe komt de prijsvorming tot stand?,https://radar.avrotros.nl/forum/energie-f51/de...,22 okt 2021 14:18,197725,48
"Woningen met verkeerde energielabels, wie heeft daar ervaring mee?",https://radar.avrotros.nl/forum/energie-f51/wo...,13 feb 2023 19:48,200283,32
slimme meter meet niet altijd correct,https://radar.avrotros.nl/forum/energie-f51/sl...,14 feb 2023 11:37,200287,3
Energieleverancier kiezen met zonnepanelen,https://radar.avrotros.nl/forum/energie-f51/en...,05 jan 2021 16:57,195768,100
Hogere tarieven Vattenfall ontlopen,https://radar.avrotros.nl/forum/energie-f51/ho...,23 dec 2022 16:42,199987,9
...,...,...,...,...
Een energiecontract zonder mijn toestemming?,https://radar.avrotros.nl/forum/energie-f51/ee...,09 mei 2022 17:04,198866,6
Vattenfall wil Warmtelink plaatsen voor digitaal uitlezen verbruik,https://radar.avrotros.nl/forum/energie-f51/va...,02 jun 2021 17:18,196961,47
Energierekening opsplitsen in Netbeheer en Levering,https://radar.avrotros.nl/forum/energie-f51/en...,03 mar 2022 09:41,198540,17
Faillisement Energie Welkom: wat kan / mag ik?,https://radar.avrotros.nl/forum/energie-f51/fa...,15 nov 2021 09:41,197882,31


In [2]:
# Get first ten rows within df
topics = df_topics.head(2)


dict_messages = {}
message_id = 1

for topic, row in topics.iterrows():
    page_index = 0
    while True:
        if page_index > row['reactions']:
            break
        page_link = "https://radar.avrotros.nl/forum/viewtopic.php?t=" + row['code'] + "&start=" + str(page_index)
        print(page_link)
        page = requests.get(page_link)
        if page.status_code != 200:
            break
        soup = BeautifulSoup(page.content, 'html.parser')
        posts = soup.find_all('div', class_=lambda x: x and x.startswith("post has-profile bg"))
        for post in posts:
            content = post.find("div", class_="content")
            # Remove blockquotes to prevent double counting
            for blockquote in content.find_all("blockquote"):
                blockquote.decompose()
            content = content.text

            date = post.find('p', class_="author").find('time').text

            dict_messages[message_id] = topic, content, date
            message_id += 1

        page_index += 20


df_messages = pd.DataFrame.from_dict(dict_messages, orient='index', columns=['topic', 'content', 'date'])

# Removing newlines and tabs from content cells.
df_messages['content'] = df_messages['content'].str.replace('\n', '')
df_messages['content'] = df_messages['content'].str.replace('\t', '')

# Convert dates to datetime
from datetime import datetime
import locale
locale.setlocale(locale.LC_ALL, "nl_NL")

df_messages["date"] = df_messages["date"].str.split(" ").apply(lambda x: " ".join([i.capitalize() for i in x]))

print(datetime.strptime("22 Okt 2021 14:32", "%d %b %Y %H:%M"))

df_messages['date'] = df_messages['date'].apply(lambda x: datetime.strptime(x, "%d %b %Y %H:%M"))

df_messages



https://radar.avrotros.nl/forum/viewtopic.php?t=197725&start=0
https://radar.avrotros.nl/forum/viewtopic.php?t=197725&start=20
https://radar.avrotros.nl/forum/viewtopic.php?t=197725&start=40
https://radar.avrotros.nl/forum/viewtopic.php?t=200283&start=0
https://radar.avrotros.nl/forum/viewtopic.php?t=200283&start=20
2021-10-22 14:32:00


,topic,content,date
1,De energiemarkt en salderen: hoe komt de prijs...,"Goedendag,In navolging van het inmiddels geslo...",2021-10-22 14:18:00
2,De energiemarkt en salderen: hoe komt de prijs...,3.U verbruikt en produceert stroomDezelfde pri...,2021-10-22 14:52:00
3,De energiemarkt en salderen: hoe komt de prijs...,Heel leerzaam artikel.,2021-10-22 17:23:00
4,De energiemarkt en salderen: hoe komt de prijs...,Dank voor de uitgebreide uitleg.,2021-10-22 17:57:00
5,De energiemarkt en salderen: hoe komt de prijs...,Ik heb de mods gevraagd dit topic sticky te ma...,2021-10-22 18:11:00
...,...,...,...
78,"Woningen met verkeerde energielabels, wie heef...",Alweer een statement wat niet is onderbouwd. J...,2023-02-15 08:08:00
79,"Woningen met verkeerde energielabels, wie heef...",De labels die de bewoners kregen waren geen of...,2023-02-15 08:35:00
80,"Woningen met verkeerde energielabels, wie heef...",Zit er een kras op uw plaat? Kom dan eens met ...,2023-02-15 08:55:00
81,"Woningen met verkeerde energielabels, wie heef...",Dan heb je er dus niks aan. Niet om de hoogte ...,2023-02-15 09:07:00


In [5]:
print(df_messages['content'])

for i in df_messages['content']:
    print(type(i))

1     Goedendag,In navolging van het inmiddels geslo...
2     3.U verbruikt en produceert stroomDezelfde pri...
3                                Heel leerzaam artikel.
4                      Dank voor de uitgebreide uitleg.
5     Ik heb de mods gevraagd dit topic sticky te ma...
                            ...                        
78    Alweer een statement wat niet is onderbouwd. J...
79    De labels die de bewoners kregen waren geen of...
80    Zit er een kras op uw plaat? Kom dan eens met ...
81    Dan heb je er dus niks aan. Niet om de hoogte ...
82    Voor mij is dat duidelijk uit de bijdragen te ...
Name: content, Length: 82, dtype: object
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 

In [6]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

def sentiment_determinator(message):
    sentiment = classifier(message)
    return sentiment[0]['label']


def apply_sentiment_determinator(row):
    sentiment = sentiment_determinator(row["content"])
    return sentiment

# apply the function to each cell in the "content" column and store the result in the "sentiment" column
df_messages["sentiment"] = df_messages.apply(apply_sentiment_determinator, axis=1)

df_messages

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.